<a href="https://colab.research.google.com/github/RanaMudassarRasool/DeeplearningForMedicalImages/blob/main/Chest_Diseases_by_Medical_Imaging_and_Cough_Sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chest Diseases by Medical Imaging and Cough Sounds

About Dataset
This dataset contains cough sound images (CSI), chest X-rays (CXR), and CT scan images of several chest diseases such as COVID-19, lung cancer (LC), consolidation lung (COL), atelectasis (ATE), tuberculosis (TB), pneumothorax (PNEUTH), edema (EDE), pneumonia (PNEU) and normal (NOR). The dataset was collected from Al-Shafa Hospital, Multan, Pakistan in collaboration with Apple Pharma medicine dealer company. The statistics of the dataset are presented in Table 1. The meta-information of the patient is not provided. However, the dataset is about 20% of female and 80% of male CSI, CXR, and CT scan images. In addition, CXR, CSI, and CT scans of individuals aged 41 to 62 years were collected.

Table 1. Statistics of the chest diseases dataset.

Sr#	Chest Diseases	CSI	CXR	CT
1	COVID-19	28	5	17
2	Lungs Cancer	30	7	5
3	Consolidation Lung	18	5	3
4	Atelectasis	25	4	4
5	Tuberculosis	27	4	6
6	Pneumothorax	18	5	4
7	Edema	18	5	9
8	Pneumonia	16	10	15
9	Normal	24	6	8
Malik, Hassaan; Anees, Tayyaba (2023), “Chest Diseases Using Different Medical Imaging and Cough Sounds”, Mendeley Data, V1, doi: 10.17632/y6dvssx73b.1

In [12]:
# Importing Libraries
from tensorflow import keras
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers, losses
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image

import pickle
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [56]:
# Base Path for all files
data_dir = '/content/drive/MyDrive/ChestDiseasesUsingDifferentMedicalImagingandCoughSounds/Chest Diseases Dataset/Chest Diseases Dataset'

**Load Images For Training & Testing**

In [75]:
###### Using ImageDataGenerator to load the Images for Training and Testing the CNN Model
datagenerator = {
    "train": ImageDataGenerator(horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1. / 255,
                                validation_split=0.1,
                                shear_range=0.1,
                                zoom_range=0.1,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                rotation_range=30,
                               ).flow_from_directory(directory=data_dir,
                                                     target_size=(256, 256),
                                                     subset='training',
                                                    ),

    "valid": ImageDataGenerator(rescale=1 / 255,
                                validation_split=0.1,
                               ).flow_from_directory(directory=data_dir,
                                                     target_size=(256, 256),
                                                     subset='validation',
                                                    ),
"test": ImageDataGenerator(rescale=1 / 255)
    }

Found 297 images belonging to 9 classes.
Found 29 images belonging to 9 classes.


**Initialize Densenet-121 Pretrained Model**

In [66]:
# Initializing InceptionV3 (pretrained) model with input image shape as (300, 300, 3)
base_model = DenseNet121(weights=None, include_top=False, input_shape=(256, 256, 3))

# Load Weights for the InceptionV3 Model
base_model.load_weights('/content/drive/MyDrive/DenseNet-BC-121-32-no-top.h5')

# Setting the Training of all layers of InceptionV3 model to false
base_model.trainable = False

In [76]:
# Define the model architecture
model = Sequential()

# Load pre-trained DenseNet121 model (without the final classification layers)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
  layer.trainable = False

# Add additional layers for classification
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))  # Dense layer with 1024 neurons and ReLU activation
model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
model.add(Dense(9, activation='softmax'))  # Output layer with 9 neurons (one for each class) and softmax activation

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(datagenerator['train'], epochs=10, validation_data=datagenerator['valid'])


29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/10
10/10 [==============================] - 102s 9s/step - loss: 2.5759 - accuracy: 0.1717 - val_loss: 1.9242 - val_accuracy: 0.3448
Epoch 2/10
10/10 [==============================] - 93s 9s/step - loss: 2.1901 - accuracy: 0.2256 - val_loss: 1.5935 - val_accuracy: 0.4138
Epoch 3/10
10/10 [==============================] - 92s 9s/step - loss: 2.0702 - accuracy: 0.2290 - val_loss: 1.5135 - val_accuracy: 0.3793
Epoch 4/10
10/10 [==============================] - 91s 9s/step - loss: 1.8755 - accuracy: 0.3064 - val_loss: 1.5685 - val_accuracy: 0.4138
Epoch 5/10
10/10 [==============================] - 92s 9s/step - loss: 1.7710 - accuracy: 0.3300 - val_loss: 1.4020 - val_accuracy: 0.3103
Epoch 6/10
10/10 [==============================] - 92s 9s/step - loss: 1.7254 - accuracy: 0.3030 - val_loss: 1.4609 - val_accuracy: 0.2759
Epoch 7/10
10/10 [==============================] - 91s 10s/step - loss: 1.6086 - accuracy: 0.

In [80]:
# Function to predict on a single image
def predict_on_image(img_path):
  # Load the image and pre-process it
  img = image.load_img(img_path, target_size=(256, 256))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)  # Add a dimension for batch processing
  x = x / 255.0  # Rescale to 0-1

  # Predict using the trained model
  prediction = model.predict(x)[0]

  # Get the class with the highest probability
  predicted_class_idx = np.argmax(prediction)
  predicted_class = class_labels[predicted_class_idx]

  # Print the predicted class and probability of each class
  print(f"Predicted class: {predicted_class}")
  print(f"Probabilities: {prediction}")

# Replace 'class_labels' with the actual class labels from your data
class_labels = ['Normal', 'Pneumonia', 'Edema', 'Pneumothorax', 'Tuberculosis',
                'Tuberculosis', 'Atelectasis', 'Consolidation Lung', 'COVID-19']

# Get the path to the image you want to predict on
predict_on_image("/content/drive/MyDrive/ChestDiseasesUsingDifferentMedicalImagingandCoughSounds/Chest Diseases Dataset/Chest Diseases Dataset/1. COVID-19/CT Scan/Image 01 (1).jpeg")




1/1 [==============================] - 0s 247ms/step
Predicted class: Consolidation Lung
Probabilities: [0.07258204 0.07999437 0.10200949 0.0802157  0.1402798  0.09301724
 0.07643994 0.2528566  0.10260493]
